In [1]:
from bs4 import BeautifulSoup
import glob
import numpy as np
import pandas as pd
import re
import sys

# Fixing CT 2023 edge cases

In [2]:
dfs = []
for i in range(6):
    dfs.append(pd.read_csv(f"ct_last_urls_{i}_processed.csv", index_col=0))

df = pd.concat(dfs)

In [3]:
df[df.isna().any(axis=1) & df['link']]

,title,source,date,text,link
326941,NaN,NaN,NaN,NaN,https://www.chinatimes.com/amp/opinion/2023040...
327379,NaN,NaN,NaN,NaN,https://www.chinatimes.com/amp/realtimenews/20...
329412,NaN,NaN,NaN,NaN,https://www.chinatimes.com/amp/realtimenews/20...
336647,NaN,NaN,NaN,NaN,https://www.chinatimes.com/amp/realtimenews/20...
679446,2023年10月份PEUGEOT最新購車優惠專案,旺車,2023/10/02,NaN,https://www.chinatimes.com/amp/realtimenews/20...
679446,2023年10月份PEUGEOT最新購車優惠專案,旺車,2023/10/02,NaN,https://www.chinatimes.com/amp/realtimenews/20...


In [30]:
df = df.drop([679446, 35720], axis=0) # Article without a body


In [31]:
df = df.drop_duplicates('link')

In [37]:
df = df.drop_duplicates(["title"]).sort_index()

In [48]:
df[pd.to_datetime(df['date']) < "2024-02-01"].to_parquet("ct_2023_contents.parquet")

/Users/ethanelasky/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


# Fixing CT 2020 Edge cases

In [ ]:
# This is in addition to the above generic preprocessing
df = df.drop([7259413, 7403343, 6916026, 7048387, 7168086, 7314819, 7619206, 8015108]) # Dropping photo album entries (圖輯)

# All together

In [5]:
import pandas as pd
dfs = []
dfs.append(pd.read_parquet('ct_2020_contents.parquet'))
dfs.append(pd.read_parquet("ct_2021_contents.parquet"))
dfs.append(pd.read_parquet('ct_2022_contents.parquet'))
dfs.append(pd.read_parquet('ct_2023_contents.parquet'))

In [8]:
df = pd.concat(dfs).drop_duplicates('title')

In [21]:
df = df[df['video_only'] != True]
df = df.drop('video_only', axis=1)
df['paper'] = df['source']
df['source'] = 'ct'

In [24]:
df.to_parquet("ct_contents.parquet")